In [24]:
import pandas as pd
#import os
#pip install openpyxl
import numpy as np
import scipy as sp

In [109]:
Data=pd.read_excel("DataCS-2018-2021.xlsx",
                   sheet_name=None,
                   usecols="B:P",
                   skipfooter=15,
                   header=7)



#LOADING SHEETS
Data18=Data['2018']
Data19=Data['2019']
Data20=Data['2020']
Data21=Data['2021']

#Data.keys()
#RENAME

# RENAME COLUMNS
new_columns = {0: 'Ma CK', 
               1: 'Ten Cong ty',
               2: 'San',
               3: 'Nhom nganh',
               4: 'Tong Tai san',
               5: 'Tong No phai tra',
               6: 'Von Chu so huu',
               7: 'No ngan han',
               8: 'No dai han',
               9: 'Tai san co dinh',
               10: 'Gia von hang ban',
               11: 'Loi nhuan truoc thue',
               12: 'Chi phi lai vay',
               13: 'Von hoa thi truong',
               14: 'ROA'               
              }

#RENAME and DROP ROWS

Data18=Data18.rename(columns={Data18.columns[i]: new_col for i, new_col in new_columns.items()})
Data19=Data19.rename(columns={Data19.columns[i]: new_col for i, new_col in new_columns.items()})
Data20=Data20.rename(columns={Data20.columns[i]: new_col for i, new_col in new_columns.items()})
Data21=Data21.rename(columns={Data21.columns[i]: new_col for i, new_col in new_columns.items()})

#DROP ROW 1-2
Data18.drop([0,1], axis=0, inplace=True)
Data19.drop([0,1], axis=0, inplace=True)
Data20.drop([0,1], axis=0, inplace=True)
Data21.drop([0,1], axis=0, inplace=True)

# LOC trong San Giao dich HOSE HNX UPCOM
San_GD = ['HOSE','HNX','UPCOM']
Data18 = Data18[Data18['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data19 = Data19[Data19['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data20 = Data20[Data20['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data21 = Data21[Data21['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)



In [110]:
# FILTER : TONG TAI SAN > 100B; Von Hoa thi truong > 100B
Data18=Data18[(Data18['Tong Tai san'] >= 100*10**9) & (Data18['Von hoa thi truong'] > 100 )]


,Ma CK,Ten Cong ty,San,Nhom nganh,Tong Tai san,Tong No phai tra,Von Chu so huu,No ngan han,No dai han,Tai san co dinh,Gia von hang ban,Loi nhuan truoc thue,Chi phi lai vay,Von hoa thi truong,ROA
14,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,53624854042612,22838548658743,30786305383869,7683984257773,15154564400970,18447261180067,-8295605070971,7618175620385,-95084582355,186157.562634,0.05359
10,MSN,Tập đoàn Masan,HOSE,Hàng Tiêu dùng L1,64578613000000,30498935000000,34079678000000,15795515000000,14703420000000,29203764000000,-26306208000000,6243824000000,-2667531000000,152623.296738,0.081446
31,VNM,VINAMILK,HOSE,Hàng Tiêu dùng L1,37366108654179,11094739362252,26271369291927,10639592009462,455147352790,13365353599098,-27950543501501,12051696266123,-51367418852,150267.796495,0.178484
7,HPG,Hòa Phát,HOSE,Nguyên vật liệu L1,78223007670925,37600057830115,40622949840810,22636149492136,14963908337979,12782560625001,-44165626148685,10071072872302,-539861243640,139264.117515,0.160962
51,SAB,SABECO,HOSE,Hàng Tiêu dùng L1,22366742792213,6254837224044,16111905568169,5925696932383,329140291661,4567091079194,-27864413389551,5390439641506,-35244808353,116713.175852,0.153027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251,DHN,Dược Hà Nội,UPCOM,Dược phẩm và Y tế L1,100198865212,26288021686,73910843526,26288021686,0,34576380025,-47882027395,8447985575,0,103.17076,NaN
587,PXT,Xây lắp Đường ống Dầu khí,UPCOM,Công nghiệp L1,394370343715,291110736138,103259607577,286255735637,4855000501,57878219837,-149914305074,1966662415,-6411295666,101.74,-0.016945
1250,EBS,Sách Giáo dục Hà Nội,HNX,Dịch vụ Tiêu dùng L1,154083015581,26401318901,127681696680,26355318901,46000000,3322075035,-129290391139,15021319057,-21528488,101.648957,0.081107
790,PSC,Vận tải Petrolimex SG,HNX,Công nghiệp L1,274842483387,150621629271,124220854116,68955376074,81666253197,176909495601,-456812227535,18902774491,-4713525973,101.52,0.035081


In [119]:
# Data18.groupby('Nhom nganh').mean()
Data18.describe().transpose()

,count,unique,top,freq
Ma CK,1321,1317,MVC,2
Ten Cong ty,1321,1316,XNK Thiên Nam,2
San,1321,3,UPCOM,756
Nhom nganh,1321,7,Công nghiệp L1,634
Tong Tai san,1211,1207,1507453072979,2
Tong No phai tra,1211,1207,363936433192,2
Von Chu so huu,1211,1207,1143516639787,2
No ngan han,1211,1207,362903439786,2
No dai han,1211,935,0,265
Tai san co dinh,1211,1193,0,15
